In [57]:
import pandas as pd
from Levenshtein import distance

laces_file = 'data/Laces Students.csv'
ged_file = 'data/GED test takers.csv'

In [44]:
l = pd.read_csv(laces_file, parse_dates=['Birthdate'])
if 'SSN' in l.columns.tolist():
    print(f"Dropping PII")
    l = l.drop(columns=['SSN'])
    l.to_csv(laces_file, header=True, index=False)

In [162]:
l['DDB'] = l['Birthdate'].dt.strftime('%Y-%m-%d')
l[['DDB', 'Birthdate']]

,DDB,Birthdate
0,1956-08-03,1956-08-03
1,1969-12-13,1969-12-13
2,1995-07-11,1995-07-11
3,1983-12-11,1983-12-11
4,1990-06-20,1990-06-20
...,...,...
13023,1992-01-30,1992-01-30
13024,1989-03-05,1989-03-05
13025,1966-12-12,1966-12-12
13026,1981-09-03,1981-09-03


In [45]:
l, g = pd.read_csv(laces_file, parse_dates=['Birthdate']), pd.read_csv(ged_file, parse_dates=['Date of Birth'])
l = l.drop_duplicates()
g = g.drop_duplicates()

In [59]:
for c in ['Last Name', 'First Name']:
    g.loc[:, c] = g[c].apply(lambda n: n.lower())
    l.loc[:, c] = l[c].apply(lambda n: n.lower())

In [64]:
g.loc[:, 'Key'] = g[['Last Name', 'First Name', 'DOB']].agg(tuple, axis=1)
l.loc[:, 'Key'] = l[['Last Name', 'First Name', 'DOB']].agg(tuple, axis=1)

In [65]:
g['Key']

0      (johnson, katelyn, 1996-04-03)
1        (ortiz, britney, 1993-03-21)
2      (huggins, patrick, 2003-05-30)
3        (eddings, lamar, 1974-01-29)
4        (strunk, joshua, 1983-09-20)
                    ...              
925     (stevenson, dior, 1994-09-03)
926     (stevenson, dior, 1994-09-03)
927     (stevenson, dior, 1994-09-03)
928        (meade, peter, 1993-07-26)
929        (meade, peter, 1993-07-26)
Name: Key, Length: 928, dtype: object

In [66]:
l['Key']

0                   (aaron, bertha, 1956-08-03)
1              (aaron, teralawanda, 1969-12-13)
2                 (aarons, brianna, 1995-07-11)
3                   (aarons, jimmy, 1983-12-11)
4                 (abbey, shanetta, 1990-06-20)
                          ...                  
13023           (zorrilla, natalie, 1992-01-30)
13024               (zuber, tamiko, 1989-03-05)
13025    (zuber-jackson, stephanie, 1966-12-12)
13026              (zunic, william, 1981-09-03)
13027           (zwolenik, raymond, 1992-01-12)
Name: Key, Length: 13028, dtype: object

In [ ]:
laces_keys = l[['Last Name', 'First Name', 'DOB']].agg(tuple, axis=1)
# laces_keys

In [180]:
geds_keys = g[['Last Name', 'First Name', 'DOB']].agg(tuple, axis=1).tolist()
geds_keys

[('johnson', 'katelyn', '1996-04-03'),
 ('ortiz', 'britney', '1993-03-21'),
 ('huggins', 'patrick', '2003-05-30'),
 ('eddings', 'lamar', '1974-01-29'),
 ('strunk', 'joshua', '1983-09-20'),
 ('daniels', 'paula', '1999-05-14'),
 ('root', 'sarah', '1990-08-27'),
 ('morris', 'krystalyn', '1990-04-01'),
 ('kyer', 'brandy', '1991-04-11'),
 ('garrett', 'jessica', '1989-07-24'),
 ('jones', 'luther', '1994-02-25'),
 ('stephens', 'james', '2000-05-18'),
 ('mclaughlin', 'chiffon', '1982-08-26'),
 ('johnson', 'jessica', '1992-06-02'),
 ('robinson', 'tahja', '2000-01-13'),
 ('armstrong', 'jessica', '1979-05-15'),
 ('webber', 'kyle', '1996-07-08'),
 ('brittany', 'thompson', '1986-10-14'),
 ('johnson', 'gabrielle', '1991-02-05'),
 ('cofield', 'emanuel', '1976-06-07'),
 ('davis', 'makala', '1996-01-16'),
 ('bryant', 'luci', '1993-02-16'),
 ('binegar', 'sidney', '1996-12-16'),
 ('eddings', 'lamar', '1974-01-29'),
 ('greggs', 'korissa', '1998-09-28'),
 ('greggs', 'korissa', '1998-09-28'),
 ('greggs', 'k

In [187]:
def _lev(x):
    rates = [sum([distance(n, m) for n, m in zip(x, gm)]) for gm in geds_keys]
    rate_min = min(rates)
    return rate_min, geds_keys[rates.index(rate_min)]

ds = pd.DataFrame([tuple(x) + _lev(x) for x in laces_keys], columns=['Last', 'First', 'DoB', 'Distance', 'Closest Match'])
ds

,Last,First,DoB,Distance,Closest Match
0,aaron,bertha,1956-08-03,10,"(boone, briana, 1998-08-03)"
1,aaron,teralawanda,1969-12-13,14,"(crowe, arlando, 1979-01-10)"
2,aarons,brianna,1995-07-11,9,"(hershey, brianna, 1999-03-17)"
3,aarons,jimmy,1983-12-11,10,"(parsons, megan, 1988-12-20)"
4,abbey,shanetta,1990-06-20,10,"(martin, shannita, 1991-02-10)"
...,...,...,...,...,...
13023,zorrilla,natalie,1992-01-30,11,"(portillo, bryant, 1992-01-23)"
13024,zuber,tamiko,1989-03-05,10,"(luke, omika, 1986-07-14)"
13025,zuber-jackson,stephanie,1966-12-12,14,"(ryon, stephanie, 1991-04-12)"
13026,zunic,william,1981-09-03,9,"(daroche, william, 1952-08-03)"


In [189]:
ds.loc[ds.Distance < 5.0].sort_values('Distance')

,Last,First,DoB,Distance,Closest Match
143,alexander,tameka,1979-09-02,0,"(alexander, tameka, 1979-09-02)"
12386,wideman,maranda,1985-08-31,0,"(wideman, maranda, 1985-08-31)"
12183,webber,kyle,1996-07-08,0,"(webber, kyle, 1996-07-08)"
11765,vanadia,alexandrea,1991-10-25,0,"(vanadia, alexandrea, 1991-10-25)"
10138,sani,abdullah,1983-03-15,0,"(sani, abdullah, 1983-03-15)"
9999,russ,heather,1995-12-25,0,"(russ, heather, 1995-12-25)"
9502,rivera,erika,1988-02-23,0,"(rivera, erika, 1988-02-23)"
9419,rice,andre,1976-08-22,0,"(rice, andre, 1976-08-22)"
8469,pagan,maricenia,1974-03-18,0,"(pagan, maricenia, 1974-03-18)"
8350,ortiz,britney,1993-03-21,0,"(ortiz, britney, 1993-03-21)"
